In [1]:
import numpy as np

def readMatrix(file):
    fd = open(file, 'r')
    hdr = fd.readline()
    rows, cols = [int(s) for s in fd.readline().strip().split()]
    tokens = fd.readline().strip().split()
    matrix = np.zeros((rows, cols))
    Y = []
    for i, line in enumerate(fd):
        nums = [int(x) for x in line.strip().split()]
        Y.append(nums[0])
        kv = np.array(nums[1:])
        k = np.cumsum(kv[:-1:2])
        v = kv[1::2]
        matrix[i, k] = v
    category = (np.array(Y) * 2) - 1
    return matrix, tokens, category

In [90]:
def svm_train(matrix, category, tau = 8, outer_loops = 40):
    state = {}
    M, N = matrix.shape
    #####################
    Y = category
    matrix = 1. * (matrix > 0)
    squared = np.sum(matrix, axis=1)
    gram = matrix.dot(matrix.T)
    K = np.exp(-(squared.reshape((1, -1)) + squared.reshape((-1, 1)) - 2 * gram) / (2 * (tau ** 2)) )

    alpha = np.zeros(M)
    alpha_avg = np.zeros(M)
    L = 1. / (tau ** 2 * M)

    alpha_avg
    for ii in range(outer_loops * M):
        i = int(np.random.rand() * M)
        margin = Y[i] * np.dot(K[i, :], alpha)
        grad = M * L * K[:, i] * alpha[i]
        if (margin < 1):
            grad -=  Y[i] * K[:, i]
        alpha -=  grad / np.sqrt(ii + 1)
        alpha_avg += alpha

    alpha_avg /= (ii + 1) * M

    state['alpha'] = alpha
    state['alpha_avg'] = alpha_avg
    state['Xtrain'] = matrix
    state['Sqtrain'] = squared
    ####################
    return state

In [16]:
def svm_test(matrix, state, tau = 8):
    M, N = matrix.shape
    output = np.zeros(M)
    ###################
    Xtrain = state['Xtrain']
    Sqtrain = state['Sqtrain']
    matrix = 1. * (matrix > 0)
    squared = np.sum(matrix * matrix, axis=1)
    gram = matrix.dot(Xtrain.T)
    K = np.exp(-(squared.reshape((-1, 1)) + Sqtrain.reshape((1, -1)) - 2 * gram) / (2 * (tau ** 2)))
    alpha_avg = state['alpha_avg']
    preds = K.dot(alpha_avg)
    output = np.sign(preds)
    ###################
    return output

In [5]:
def evaluate(output, label):
    error = (output != label).sum() * 1. / len(output)
    print ('Error: %1.4f' % error)

In [88]:
def main():
    trainMatrix, tokenlist, trainCategory = readMatrix('dataset/MATRIX.TRAIN.400')
    testMatrix, tokenlist, testCategory = readMatrix('dataset/MATRIX.TEST')

    state = svm_train(trainMatrix, trainCategory)
    output = svm_test(testMatrix, state)

    evaluate(output, testCategory)
    return

In [105]:
def perception_in_primal(X, Y, epoch = 40):
    M, N = X.shape
    theta = np.random.randn(N)
    for i in range(epoch):
        for j in range(M):
            margin = Y[j] * theta.dot(X[j, :].T)
            if margin <= 0:
                theta += Y[j] * X[j, :]
    print(test_perception_primal(X, Y, theta))
    return theta

In [106]:
def test_perception_primal(X, Y, theta):
    h = X @ theta
    return h[h * Y < 0].size / Y.size

In [ ]:
def perception_in_dual(X, Y, epoch = 40):
    M, N = X.shape
    theta = np.random.randn(N)
    for i in range(epoch):
        for j in range(M):
            margin = Y[j] * theta.dot(X[j, :].T)
            if margin <= 0:
                theta += Y[j] * X[j, :]
    print(test_perception_dual(X, Y, theta))
    return theta

In [92]:
def test_perception_dual(X, Y, K, alpha):
    pass

In [122]:
trainMatrix, tokenlist, trainCategory = readMatrix('dataset/MATRIX.TRAIN.400')
testMatrix, tokenlist, testCategory = readMatrix('dataset/MATRIX.TEST')
print(theta)
theta = perception_in_primal(trainMatrix, trainCategory)
test_perception_primal(testMatrix, testCategory, theta)

[-0.06269974  4.47015686  0.75406614 ...  0.65284343  0.79165608
  1.37193174]
0.0


0.0125

In [89]:
main()

(400, 400)
Error: 0.0025
